In [1]:
using LinearAlgebra
using Plots

In [15]:

function forward_substitution(L, b)
    x = similar(b)
    
    x[1] = b[1] / L[1,1]
    for k = 2:length(b)
        x[k] = b[k]
        
        for j = 1:k-1
            x[k] -= L[k,j] * x[j]
        end
        
        x[k] /= L[k,k]
    end
    
    return x
end


function backward_substitution(L, b)
    l = length(b)
    x = similar(b)
    
    x[end] = b[end] / L[end,end]
    for k = (l - 1):-1:1
        x[k] = b[k] 
        
        for j = k+1:l
            x[k] -= L[k,j] * x[j]
        end
        # - view(L,j,j+1:l)' * view(x,j+1:l)) 
        x[k] /= L[k,k]
    end
    return x
end



struct LU_Fac
    L::UnitLowerTriangular
    U::UpperTriangular
end



function lu_factorization!(A)
    n = min(size(A)...)
  
    @inbounds begin
        for k = 1:n-1
            Akkinv = inv(A[k,k])
            for j = k+1:n
                A[j,k] *= Akkinv
            end
            for l = k+1:n
                for j = k+1:n
                    A[j,l] -= A[j,k] * A[k,l]
                end
            end
        end
    end
    
    return LU_Fac(
        UnitLowerTriangular(A),
        UpperTriangular(A)
    )
end


lu_factorization(A) = lu_factorization!(copy(A))


function lu_solve(A::LU_Fac, b)
    y = forward_substitution(A.L, b)
    x = backward_substitution(A.U, y)
end

function lu_solve(A::AbstractArray, b)
    lu = LU_Fac(A)
    y = forward_substitution(lu.L, b)
    x = backward_substitution(lu.U, y)
end

import Base: \, show
\(A::LU_Fac, b::AbstractArray) = lu_solve(A, b);

function show(io::IO, mime::MIME{Symbol("text/plain")}, F::LU_Fac)
    print(io, "L = ")
    show(io, mime, F.L)
    print(io, "\n\nU = ")
    show(io, mime, F.U)
end


show (generic function with 346 methods)

In [20]:
A = rand(1000, 1000)
@time fac = lu_factorization(A)
@time LinearAlgebra.generic_lufact!(copy(A))
display(fac)
fac.L * fac.U ≈ A

  0.467779 seconds (9 allocations: 7.630 MiB, 0.55% gc time)
 

L = 1000×1000 UnitLowerTriangular{Float64,Array{Float64,2}}:
 1.0          ⋅          ⋅          ⋅         …    ⋅          ⋅          ⋅ 
 0.391856    1.0         ⋅          ⋅              ⋅          ⋅          ⋅ 
 0.444234   -2.94545    1.0         ⋅              ⋅          ⋅          ⋅ 
 0.873651    1.97802   -0.647755   1.0             ⋅          ⋅          ⋅ 
 0.189051   -4.50246    1.85978    0.804959        ⋅          ⋅          ⋅ 
 0.585381   -4.54483    2.67025    1.0563     …    ⋅          ⋅          ⋅ 
 0.711026   -0.710272   0.869939   0.130086        ⋅          ⋅          ⋅ 
 0.833569    0.359511   1.03871    1.60249         ⋅          ⋅          ⋅ 
 1.18048     0.251004   0.935158   0.873703        ⋅          ⋅          ⋅ 
 1.22424    -2.67474    0.632518  -0.704943        ⋅          ⋅          ⋅ 
 1.08113     1.83463   -0.705207  -1.34792    …    ⋅          ⋅          ⋅ 
 0.182391   -0.251215  -1.67564    0.44039         ⋅          ⋅          ⋅ 
 0.498345   -0.323534  -1.8

 0.449111 seconds (8 allocations: 7.637 MiB)


true

In [23]:
A = UnitLowerTriangular(rand(1000, 1000))
b = A * ones(1000)
@time forward_substitution(A, b)

  0.003931 seconds (5 allocations: 8.094 KiB)


1000-element Array{Float64,1}:
       1.0               
       1.0               
       1.0               
       1.0000000000000002
       0.9999999999999999
       1.0               
       0.9999999999999991
       0.9999999999999999
       1.0000000000000007
       0.9999999999999992
       1.0000000000000018
       0.9999999999999991
       1.0               
       ⋮                 
 -280802.1574082712      
  178363.81123271832     
  163153.99064248113     
 -234030.98733144318     
 -517779.35657036956     
  -19354.674758464796    
   48310.83274112657     
  251647.3696774992      
  819552.2936973205      
 -276414.6149571281      
 -638574.6477370507      
  376152.61262136087     